https://github.com/bbb1991/mingkh-parser парсит сайт mingkh, мне он нужен

https://mingkh.ru/api/map/companies/78/sankt-peterburg/22116/ апи внутри сайта для получения инфы об управляющей компании

https://mingkh.ru/api/map/companies/78/ ТСЖ Санкт-Петербург

https://mingkh.ru/api/map/companies/47/ ТСЖ Ленинградской области

сайт, ссылки на который написаны выше, делает открытые вызовы к какому-то своему апи, никакой защиты там нет, максимум ограничивается количество вызовов в единицу времени, так что спарсить его можно без особых проблем, что я и сделал

In [1]:
import re
from itertools import islice
from collections import defaultdict
from itertools import islice
import json
import concurrent.futures

import requests

from tqdm import tqdm

import pandas as pd

import numpy as np

# import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
ru_stopwords = stopwords.words('russian')

from cleantext import clean

from utils import safe_str, find_street, find_house, get_url_content, apply_clean

from fastDamerauLevenshtein import damerauLevenshtein

# import pytorch_lightning as pl
# from pytorch_lightning import Trainer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vladi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


# ТСЖ из mingkh

In [3]:
# по ссылке можно получить инфу обо всех управляющих компаний, ЖКС, ТСЖ и других зарегистрированных содружеств жильцов
spb_comps_url = 'https://mingkh.ru/api/map/companies/78/'

content = get_url_content(spb_comps_url)

In [4]:
tsjs = []
for feat in tqdm(content['features']):
    tsj = {}
    tsj['id'] = feat['id']
    tsj['coords'] = feat['geometry']['coordinates']
    tsj['address'] = feat['properties']['address']
    tsj['name'] = feat['properties']['name']
    tsj['shortname'] = feat['properties']['shortname']
    tsj['clusterCaption'] = feat['properties']['clusterCaption']
    
    tsjs.append(tsj)

100%|██████████| 4451/4451 [00:00<00:00, 404569.23it/s]


In [ ]:
houses = {}

In [25]:
# здесь скачивается инфа о домах, адрес, какая управляющая компания

houses_url = 'https://mingkh.ru/api/map/houses/{}/'

def load_houses(id):
    content = get_url_content(houses_url.format(id))
    houses_list = []

    for feat in content['features']:
        house = {}
        house['coords'] = feat['geometry']['coordinates']
        house['clusterCaption'] = feat['properties']['clusterCaption']
        house['address'] = feat['properties']['address']
        house['region'] = feat['properties']['region']
        house['city'] = feat['properties']['city']

        houses_list.append(house)
    return houses_list

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    not_visited = filter(lambda x: x['id'] not in houses, tsjs)
    future_to_url = {executor.submit(load_houses, tsj['id']): tsj['id'] for tsj in not_visited}
    for future in tqdm(concurrent.futures.as_completed(future_to_url), total=len(tsjs)):
        id = future_to_url[future]
        houses[id] = future.result()

 75%|███████▌  | 3352/4451 [18:41<06:07,  2.99it/s]


In [27]:
len(houses), len(tsjs)

(4451, 4451)

In [28]:
with open('data/tsjs_spb.json', 'w', encoding='utf-8') as f:
    json.dump(tsjs, f)

with open('data/tsjs_houses_spb.json', 'w', encoding='utf-8') as f:
    json.dump(houses, f)

# ТСЖ из ВК

In [2]:
with open('data/tsjs_spb.json', 'r', encoding='utf-8') as f:
    tsjs = json.load(f)

with open('data/tsjs_houses_spb.json', 'r', encoding='utf-8') as f:
    houses = json.load(f)

In [4]:
vk_groups_df = pd.read_csv('data/jkh_groups2.csv')
vk_groups_df.head()

,id,name,description,members_count
0,117404680,"ТСЖ ""Просвещения 53-1 литер. Д""",Приглашаем в группу неравнодушных жильцов наше...,46
1,166797525,"ТСЖ ""Академическое""",NaN,15
2,60358098,Комиссия по ЖКХ Молодежной коллегии Петербурга,Комиссия по жилищно-коммунальному хозяйству Мо...,124
3,90003929,"ТСЖ ""РЖЕВКА"" пр. Косыгина 33 корп. 1","группа создана для обслуждения предложений, пр...",3
4,166842485,ТСЖ Коммуны 48 лит.А,Эта группа для всех собственников и проживающи...,15


In [5]:
vk_groups_df['name_clean'] = vk_groups_df['name'].apply(lambda x: apply_clean(safe_str(x)))
vk_groups_df['description_clean'] = vk_groups_df['description'].apply(lambda x: apply_clean(safe_str(x)))

In [6]:
vk_groups_df.head()

,id,name,description,members_count,name_clean,description_clean
0,117404680,"ТСЖ ""Просвещения 53-1 литер. Д""",Приглашаем в группу неравнодушных жильцов наше...,46,ТСЖ Просвещения 53-1 литер. Д,Приглашаем в группу неравнодушных жильцов наше...
1,166797525,"ТСЖ ""Академическое""",NaN,15,ТСЖ Академическое,
2,60358098,Комиссия по ЖКХ Молодежной коллегии Петербурга,Комиссия по жилищно-коммунальному хозяйству Мо...,124,Комиссия по ЖКХ Молодежной коллегии Петербурга,Комиссия по жилищно-коммунальному хозяйству Мо...
3,90003929,"ТСЖ ""РЖЕВКА"" пр. Косыгина 33 корп. 1","группа создана для обслуждения предложений, пр...",3,ТСЖ РЖЕВКА пр. Косыгина 33 корп. 1,"группа создана для обслуждения предложений, пр..."
4,166842485,ТСЖ Коммуны 48 лит.А,Эта группа для всех собственников и проживающи...,15,ТСЖ Коммуны 48 лит.А,Эта группа для всех собственников и проживающи...


# Сопоставление тсж из mingkh и ВК

In [11]:
tsjs_df = pd.DataFrame()

for col in ['address', 'name', 'shortname']:
    tsjs_df[f'{col}_tsj'] = [tsj[col] for tsj in tsjs]
tsjs_df[['lat', 'lon']] = [tsj['coords'] for tsj in tsjs]

# чищу строчки с адресом и название от кавычек и другого мусора, чтобы было удобнее их сопоставить
tsjs_df['name_tsj_clean'] = tsjs_df['name_tsj'].apply(lambda x: apply_clean(safe_str(x)))
tsjs_df['shortname_tsj_clean'] = tsjs_df['shortname_tsj'].apply(lambda x: apply_clean(safe_str(x)))

tsjs_df

,address_tsj,name_tsj,shortname_tsj,lat,lon,name_tsj_clean,shortname_tsj_clean
0,"Санкт-Петербург г, пр-кт. Суворовский, д. 60, ...",Управляющая компания «ЖКС №3 Центрального района»,УК «ЖКС №3 Центрального района»,59.945378,30.387814,Управляющая компания ЖКС №3 Центрального района,УК ЖКС №3 Центрального района
1,"Санкт-Петербург г, пр-кт. Английский",Управляющая компания «Жкс №1 адмиралтейского р...,УК «Жкс №1 адмиралтейского района»,59.917533,30.290491,Управляющая компания Жкс №1 адмиралтейского ра...,УК Жкс №1 адмиралтейского района
2,"Санкт-Петербург г, ул. Достоевского",Управляющая компания «Жкс №2 центрального района»,УК «Жкс №2 центрального района»,59.926481,30.349851,Управляющая компания Жкс №2 центрального района,УК Жкс №2 центрального района
3,"Санкт-Петербург г, ул. Народная, д. 47, корп. ...",Управляющая компания «ЖКС №1 Невского района»,УК «ЖКС №1 Невского района»,59.880437,30.476945,Управляющая компания ЖКС №1 Невского района,УК ЖКС №1 Невского района
4,"ул. Моховая, д. 8, Санкт-Петербург",Управляющая компания «ЖКС1ЦР»,УК «ЖКС1ЦР»,59.945360,30.345432,Управляющая компания ЖКС1ЦР,УК ЖКС1ЦР
...,...,...,...,...,...,...,...
4446,"тер. Пулковское, ул. Переведенская, д. 4, корп...",Товарищество собственников жилья «Неоклассика»,ТСЖ «Неоклассика»,59.753323,30.372956,Товарищество собственников жилья Неоклассика,ТСЖ Неоклассика
4447,"ул. Первомайская, д. 8, литер А, кв. 72, Шушары",Товарищество собственников жилья «Первомайская 8»,ТСЖ «Первомайская 8»,59.808962,30.377007,Товарищество собственников жилья Первомайская 8,ТСЖ Первомайская 8
4448,"ул. Первомайская, д. 22, литер А, Шушары",Товарищество собственников жилья «Тележная 22»,ТСЖ «Тележная 22»,59.806767,30.371034,Товарищество собственников жилья Тележная 22,ТСЖ Тележная 22
4449,"тер. Ленсоветовский, д. 11, литер А, Шушары",Управляющая компания «Хаус»,УК «Хаус»,59.759265,30.470953,Управляющая компания Хаус,УК Хаус


In [8]:
# сопоставляю названия управляющих компаний с сайта и из вк

# я использую расстояние Дамероу, как метрика разности между строками,
# функция damerauLevenshtein возвращает нормализованное значение от 0 до 1, поэтому
# можно взять некоторое пороговое значение threshold и считать строки похожими, если
# похожесть превышает порог

# также задаю низкие веса для удаления и вставки символа, а для замен символов большие веса,
# логика такая, что замена символа меняет адрес, а вставка символа не так сильно меняет

# (хотя если подумать, то в номере дома любая замена меняет адрес, а в названии улицы это меньше влияет,
# над сопоставлением адресов я много не работал, поэтому сделано очень топорно)

name_mapper = {}
threshold = 0.85

dlev_kwargs = {
    'similarity': True,
    'deleteWeight': 1,
    'insertWeight': 1,
    'replaceWeight': 100,
    'swapWeight': 100,
}

for name in tqdm(vk_groups_df['name_clean'].unique()):
    candidates = {}
    for shortname_tsj in tsjs_df['shortname_tsj_clean'].unique():
        candidates[shortname_tsj] = damerauLevenshtein(name.lower(), shortname_tsj.lower(), **dlev_kwargs)
    candidate = max(candidates.items(), key=lambda x: x[1])
    if candidate[1] >= threshold:
        name_mapper[name] = candidate[0]
    else:
        name_mapper[name] = None


100%|██████████| 3040/3040 [08:20<00:00,  6.08it/s]


In [10]:
with open('data/name_mapper.json', 'w', encoding='utf-8') as f:
    json.dump(name_mapper, f)

In [ ]:
list(islice(name_mapper.items(), 30))

[('ТСЖ Просвещения 53-1 литер. Д', 'ТСЖ Просвещения 53-1, литер д'),
 ('ТСЖ Академическое', 'ТСЖ Академическое'),
 ('Комиссия по ЖКХ Молодежной коллегии Петербурга', None),
 ('ТСЖ РЖЕВКА пр. Косыгина 33 корп. 1', None),
 ('ТСЖ Коммуны 48 лит.А', None),
 ('Комиссия по ЧП д7/1 04.06.15', None),
 ('ТСЖ Ростовская 5 корпус 2 Славянка', None),
 ('МКД Международный Клуб Долбоёбов', None),
 ('Собственники МКД Молодежная 8-1', None),
 ('Ремонт квартир - ЖК ПАРГОЛОВО 23', None),
 ('ТСЖ Лагуна Морская набережная', None),
 ('Общество собственников двора по Лиговскому 105', None),
 ('Регистрация ТСЖ', None),
 ('ТСЖ Фомина 5-1', None),
 ('ТСЖ фомина 5-2', None),
 ('ТСЖ Художников-20', 'ТСЖ Художников 10'),
 ('Соседи | Оборонная д. 22', None),
 ('ТСЖ Наш дом - наша крепость Королева д.42кор.3', None),
 ('ДПК ДОМ УЧЁНЫХ, сообщество соседей', None),
 ('Дом Фортис- ТСЖ Фортис', None),
 ('ТСЖ Софийская 37 к 1', None),
 ('Совет дома Ириновский 33/49', None),
 ('ИМПЕРСКАЯ оппозиция!', None),
 ('Совет дома

In [12]:
houses_df = pd.DataFrame()

for col in ['address', 'clusterCaption']:
    houses_df[f'{col}_house'] = [house[col] for _, tsj_houses in houses.items() for house in tsj_houses]
houses_df[['lat_house', 'lon_house']] = [house['coords'] for _, tsj_houses in houses.items() for house in tsj_houses]

houses_df['address_house_clean'] = houses_df['address_house'].apply(lambda x: apply_clean(safe_str(x)))
houses_df['clusterCaption_house_clean'] = houses_df['clusterCaption_house'].apply(lambda x: apply_clean(safe_str(x)))

houses_df

,address_house,clusterCaption_house,lat_house,lon_house,address_house_clean,clusterCaption_house_clean
0,"Басков пер., 26 Литер А","Басков пер., 26 Литер А",59.939591,30.362580,"Басков пер., 26 Литер А","Басков пер., 26 Литер А"
1,"Басков пер., 32 Литер Б","Басков пер., 32 Литер Б",59.939447,30.364620,"Басков пер., 32 Литер Б","Басков пер., 32 Литер Б"
2,"Басков пер., 34 Литер А","Басков пер., 34 Литер А",59.939424,30.365248,"Басков пер., 34 Литер А","Басков пер., 34 Литер А"
3,"Басков пер., 35 Литер А","Басков пер., 35 Литер А",59.939311,30.362365,"Басков пер., 35 Литер А","Басков пер., 35 Литер А"
4,"Басков пер., 36 Литер А","Басков пер., 36 Литер А",59.939397,30.365689,"Басков пер., 36 Литер А","Басков пер., 36 Литер А"
...,...,...,...,...,...,...
25175,"ул. Школьная, 1 корпус 2 Литер А, Шушары","п. Шушары, ул. Школьная, 1 корпус 2 Литер А",59.808360,30.382056,"ул. Школьная, 1 корпус 2 Литер А, Шушары","п. Шушары, ул. Школьная, 1 корпус 2 Литер А"
25176,"ул. Пушкинская, 50 Литер А, Шушары","п. Шушары, ул. Пушкинская, 50 Литер А",59.805726,30.378301,"ул. Пушкинская, 50 Литер А, Шушары","п. Шушары, ул. Пушкинская, 50 Литер А"
25177,"ул. Пушкинская, 36 Литер А, Шушары","п. Шушары, ул. Пушкинская, 36 Литер А",59.804984,30.378418,"ул. Пушкинская, 36 Литер А, Шушары","п. Шушары, ул. Пушкинская, 36 Литер А"
25178,"Вилеровский пер., 6, Шушары","п. Шушары, Вилеровский пер., 6",59.811609,30.364045,"Вилеровский пер., 6, Шушары","п. Шушары, Вилеровский пер., 6"


In [16]:
# здесь уже борюсь с адресами

address_mapper = {}
threshold = 0.6

def get_candidate(name):
    dlev_kwargs = {
        'similarity': True,
        'deleteWeight': 1,
        'insertWeight': 1,
        'replaceWeight': 100,
        'swapWeight': 100,
    }
    candidates = {}
    for address_house in houses_df['address_house_clean'].unique():
        candidates[address_house] = damerauLevenshtein(name.lower(), address_house.lower(), **dlev_kwargs)
        if candidates[address_house] == 1.0:
            break
    candidate = max(candidates.items(), key=lambda x: x[1])
    return candidate

with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:
    future_to_name = {executor.submit(get_candidate, name): name for name in vk_groups_df['name_clean'].unique()}
    for future in tqdm(concurrent.futures.as_completed(future_to_name), total=len(vk_groups_df['name_clean'].unique())):
        name = future_to_name[future]
        candidate = future.result()
        if candidate[1] >= threshold:
            address_mapper[name] = candidate[0]
        else:
            address_mapper[name] = None


100%|██████████| 3040/3040 [1:33:35<00:00,  1.85s/it]  


In [17]:
with open('data/address_mapper.json', 'w', encoding='utf-8') as f:
    json.dump(address_mapper, f)

In [18]:
print(sum(1 for val in address_mapper.values() if val is not None))

286


In [19]:
list(islice(address_mapper.items(), 30))

[('ТСЖ Просвещения 53-1 литер. Д', 'пр-кт Просвещения, 41 Литер А'),
 ('ТСЖ Академическое', None),
 ('Комиссия по ЧП д7/1 04.06.15', None),
 ('ТСЖ Коммуны 48 лит.А', 'ул. Коммуны, 48 Литер А'),
 ('Регистрация ТСЖ', None),
 ('ТСЖ Ростовская 5 корпус 2 Славянка', 'ул. Брюсовская, 5 корпус 2 Литер А'),
 ('ТСЖ Фомина 5-1', None),
 ('ТСЖ РЖЕВКА пр. Косыгина 33 корп. 1', 'пр-кт Косыгина, 33 корпус 1 Литер А'),
 ('ТСЖ фомина 5-2', None),
 ('Собственники МКД Молодежная 8-1', None),
 ('Ремонт квартир - ЖК ПАРГОЛОВО 23', None),
 ('ТСЖ Художников-20', None),
 ('Комиссия по ЖКХ Молодежной коллегии Петербурга', None),
 ('МКД Международный Клуб Долбоёбов', None),
 ('ТСЖ Лагуна Морская набережная', None),
 ('Соседи | Оборонная д. 22', None),
 ('ТСЖ Софийская 37 к 1', 'ул. Софийская, 37 корпус 1 Литер А'),
 ('Дом Фортис- ТСЖ Фортис', None),
 ('Общество собственников двора по Лиговскому 105', None),
 ('afreedomlahta', None),
 ('Совет дома ЖК Невские паруса к.3', None),
 ('ТСЖ Сосновка', None),
 ('Совет

## mapping

в предыдущих ячейках я нашел подходящих кандидатов, здесь же просто вставляю их в датафрейм и получаю адрес

In [7]:
with open('data/name_mapper.json', 'r', encoding='utf-8') as f:
    name_mapper = json.load(f)

In [8]:
with open('data/address_mapper.json', 'r', encoding='utf-8') as f:
    address_mapper = json.load(f)

In [9]:
vk_groups_df['shortname_tsj_clean'] = vk_groups_df['name_clean'].map(name_mapper)
vk_groups_df['address_house_clean'] = vk_groups_df['name_clean'].map(address_mapper)

In [17]:
vk_groups_df_ = vk_groups_df.merge(tsjs_df, how='left', on='shortname_tsj_clean')
vk_groups_df_.lat.notna().sum(), vk_groups_df_.lat.isna().sum()

(372, 2776)

In [18]:
vk_groups_df_ = vk_groups_df_.merge(houses_df, how='left', on='address_house_clean')
vk_groups_df_.lat_house.notna().sum(), vk_groups_df_.lat_house.isna().sum()

(288, 2860)

In [20]:
vk_groups_df_['lat'] = vk_groups_df_['lat'].fillna(vk_groups_df_['lat_house'])
vk_groups_df_['lon'] = vk_groups_df_['lon'].fillna(vk_groups_df_['lon_house'])

In [21]:
vk_groups_df_[vk_groups_df_.lat.notna()]

,id,name,description,members_count,name_clean,description_clean,shortname_tsj_clean,address_house_clean,address_tsj,name_tsj,shortname_tsj,lat,lon,name_tsj_clean,address_house,clusterCaption_house,lat_house,lon_house,clusterCaption_house_clean
0,117404680,"ТСЖ ""Просвещения 53-1 литер. Д""",Приглашаем в группу неравнодушных жильцов наше...,46,ТСЖ Просвещения 53-1 литер. Д,Приглашаем в группу неравнодушных жильцов наше...,"ТСЖ Просвещения 53-1, литер д","пр-кт Просвещения, 41 Литер А","Санкт-Петербург г, пр-кт. Просвещения, д. 53, ...",Товарищество собственников жилья «Просвещения ...,"ТСЖ «Просвещения 53-1, литер д»",60.042923,30.380843,Товарищество собственников жилья Просвещения 5...,"пр-кт Просвещения, 41 Литер А","пр-кт Просвещения, 41 Литер А",60.047245,30.359553,"пр-кт Просвещения, 41 Литер А"
1,166797525,"ТСЖ ""Академическое""",NaN,15,ТСЖ Академическое,,ТСЖ Академическое,None,"Санкт-Петербург г, пр-кт. Гражданский, д. 79, ...",Товарищество собственников жилья «Академическое»,ТСЖ «Академическое»,60.016878,30.408502,Товарищество собственников жилья Академическое,NaN,NaN,NaN,NaN,NaN
2,166797525,"ТСЖ ""Академическое""",NaN,15,ТСЖ Академическое,,ТСЖ Академическое,None,"Санкт-Петербург г, пр-кт. Науки, д. 55, литер А",Товарищество собственников жилья «Академическое»,ТСЖ «Академическое»,60.007443,30.417360,Товарищество собственников жилья Академическое,NaN,NaN,NaN,NaN,NaN
4,90003929,"ТСЖ ""РЖЕВКА"" пр. Косыгина 33 корп. 1","группа создана для обслуждения предложений, пр...",3,ТСЖ РЖЕВКА пр. Косыгина 33 корп. 1,"группа создана для обслуждения предложений, пр...",None,"пр-кт Косыгина, 33 корпус 1 Литер А",NaN,NaN,NaN,59.944499,30.499430,NaN,"пр-кт Косыгина, 33 корпус 1 Литер А","пр-кт Косыгина, 33 корпус 1 Литер А",59.944499,30.499430,"пр-кт Косыгина, 33 корпус 1 Литер А"
5,166842485,ТСЖ Коммуны 48 лит.А,Эта группа для всех собственников и проживающи...,15,ТСЖ Коммуны 48 лит.А,Эта группа для всех собственников и проживающи...,None,"ул. Коммуны, 48 Литер А",NaN,NaN,NaN,59.958444,30.490788,NaN,"ул. Коммуны, 48 Литер А","ул. Коммуны, 48 Литер А",59.958444,30.490788,"ул. Коммуны, 48 Литер А"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,49126171,Гражданский пр. 107 корпус 1,Приветствую Вас дорогие Соседи \nДанная группа...,2,Гражданский пр. 107 корпус 1,Приветствую Вас дорогие Соседи Данная группа с...,None,"пр-кт Гражданский, 107 корпус 1 Литер А",NaN,NaN,NaN,60.028029,30.416192,NaN,"пр-кт Гражданский, 107 корпус 1 Литер А","пр-кт Гражданский, 107 корпус 1 Литер А",60.028029,30.416192,"пр-кт Гражданский, 107 корпус 1 Литер А"
3136,54356140,МКД Художников 14 Санкт-Петербург,"СОСЕДИ, ДОБРО ПОЖАЛОВАТЬ!\n\nСайт создан на до...",126,МКД Художников 14 Санкт-Петербург,"СОСЕДИ, ДОБРО ПОЖАЛОВАТЬ! Сайт создан на добро...",None,"пр-кт Художников, 14 Литер А",NaN,NaN,NaN,60.038488,30.344623,NaN,"пр-кт Художников, 14 Литер А","пр-кт Художников, 14 Литер А",60.038488,30.344623,"пр-кт Художников, 14 Литер А"
3140,54548518,"ТСЖ ""Ладожский парк 15-2""",Жизнь нашего ТСЖ,47,ТСЖ Ладожский парк 15-2,Жизнь нашего ТСЖ,ТСЖ Ладожский парк 15-2,None,"Санкт-Петербург г, ул. Латышских Стрелков, д. ...",Товарищество собственников жилья «Ладожский па...,ТСЖ «Ладожский парк 15-2»,59.930354,30.459814,Товарищество собственников жилья Ладожский пар...,NaN,NaN,NaN,NaN,NaN
3142,54243764,ТСЖ АВИА 39,"Группа создана только для жителей нашего дома,...",187,ТСЖ АВИА 39,"Группа создана только для жителей нашего дома,...",ТСЖ Авиа 39,None,"Санкт-Петербург г, пр-кт. Авиаконструкторов, д...",Товарищество собственников жилья «Авиа 39»,ТСЖ «Авиа 39»,60.021982,30.233529,Товарищество собственников жилья Авиа 39,NaN,NaN,NaN,NaN,NaN


In [24]:
vk_groups_df = vk_groups_df_

In [27]:
vk_groups_df.to_csv('data/vk_groups_addresses.csv')